In [ ]:
!tar -xvzf en_core_web_sm-3.4.0.tar.gz

In [3]:
import pandas as pd

In [4]:
df_mt = pd.read_csv("/ssd003/projects/pets/datasets/mtsamples.csv")

df_mt_X = df_mt[["description",'sample_name', 'transcription', 'keywords']]
df_mt_Y = df_mt["medical_specialty"]

In [240]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_mt_X, df_mt_Y, test_size=0.2, random_state=42)

In [241]:
X_train.describe()

,description,sample_name,transcription,keywords
count,3999,3999,3973,3145
unique,2239,2265,2246,3079
top,An example/template for a routine normal male...,Hematuria - ER Visit,"REASON FOR CONSULTATION: , Thyroid mass diagno...",
freq,10,4,4,65


In [242]:
y_train.groupby(y_train).count().sort_values(ascending=False)

medical_specialty
 Surgery                          872
 Consult - History and Phy.       409
 Cardiovascular / Pulmonary       303
 Orthopedic                       287
 General Medicine                 209
 Radiology                        205
 Gastroenterology                 188
 Neurology                        179
 SOAP / Chart / Progress Notes    139
 Obstetrics / Gynecology          125
 Urology                          121
 Discharge Summary                 87
 Neurosurgery                      76
 Hematology - Oncology             75
 ENT - Otolaryngology              73
 Nephrology                        66
 Ophthalmology                     65
 Emergency Room Reports            58
 Pediatrics - Neonatal             55
 Pain Management                   53
 Psychiatry / Psychology           46
 Podiatry                          42
 Office Notes                      40
 Dermatology                       26
 Cosmetic / Plastic Surgery        23
 Letters                        

In [243]:
import spacy

In [244]:
X_train["transcription"].iloc[0]

"EXAM:,  Mammographic screening FFDM,HISTORY: , 40-year-old female who is on oral contraceptive pills.  She has no present symptomatic complaints.  No prior history of breast surgery nor family history of breast CA.,TECHNIQUE: , Standard CC and MLO views of the breasts.,COMPARISON: , This is the patient's baseline study.,FINDINGS: , The breasts are composed of moderately to significantly dense fibroglandular tissue.  The overlying skin is unremarkable.,There are a tiny cluster of calcifications in the right breast, near the central position associated with 11:30 on a clock.,There are benign-appearing calcifications in both breasts as well as unremarkable axillary lymph nodes.,There are no spiculated masses or architectural distortion.,IMPRESSION:,  Tiny cluster of calcifications at the 11:30 position of the right breast.  Recommend additional views; spot magnification in the MLO and CC views of the right breast.,BIRADS Classification 0 - Incomplete,MAMMOGRAPHY INFORMATION:,1.  A certai

In [245]:
nlp = spacy.load("../en_core_web_sm-3.4.0/en_core_web_sm/en_core_web_sm-3.4.0/", 
                 disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

In [246]:
y_train.iloc[0]

' Radiology'

In [247]:
doc = nlp(X_train["transcription"].iloc[0])

In [255]:
# from collections import Counter
# vocab = Counter()

# def compile_vocab(s):
#     vocab.update(Counter([t.text for t in nlp(s.lower())]))
# X_train[~X_train["transcription"].isna()]["transcription"].apply(compile_vocab)
# len(vocab)

In [256]:
# import string
# with open("../stopwords/english") as fin:
#     stopwords = {l.strip() for l in fin}
    
# for s in stopwords:
#     vocab.pop(s, None)
    
# for c in string.punctuation:
#     vocab.pop(c, None)
    
# vocab.pop(' ', None)

# len(vocab)

In [257]:
# with open("./vocab_all.txt", "w", encoding="utf-8") as fout:
#     for k,c in vocab.items():
#         fout.write(f"{k}:{c}\n")

In [258]:
# from collections import Counter
# vocab = Counter()

# with open("./vocab_all.txt", encoding="utf-8") as fin:
#     for l in fin:
#         vv = l.split(':')
#         w = ':'.join(vv[:-1])
#         c = int(vv[-1])
#         vocab[w]= c

In [259]:
glove = {}
with open("../glove.6B.100d.txt", encoding="utf-8") as fin:
    for l in fin:
        vv = l.split(' ')
        v = list(map(float, vv[-100:]))
        w = ' '.join(vv[:-100])
        glove[w]=v

In [253]:
# vocab1000 = vocab.most_common(1000)
# vocab1000 = {v[0]:i for i,v in enumerate(vocab1000)}

In [260]:
import numpy as np

def word_vector(s):
    embedding = np.zeros(100)
    if not pd.isna(s):
        v = [glove[t.text] for t in nlp(s.lower()) if t.text in glove]
        embedding = np.array(v).mean(axis=0)
    return embedding

X = np.stack(X_train["transcription"].apply(word_vector))

In [262]:
X.shape

(3999, 100)

In [263]:
lbl = pd.Series(y_train.groupby(y_train).count().index)

In [264]:
def f(v):
    return lbl[lbl==v].index[0]

Y = y_train.apply(f).to_numpy()

In [279]:
Y.shape

(3999,)

In [280]:
import torch

In [281]:
class MLR(torch.nn.Module):

    def __init__(self, n_features, n_classes):
        super(MLR, self).__init__()
        self.lr = torch.nn.Linear(n_features, n_classes)
        
    def forward(self, x):
        out = torch.special.log_softmax(self.lr(x), 0)
        return out

In [298]:
model = MLR(X.shape[1], max(Y)+1)

optim = torch.optim.SGD(model.parameters(), lr=10)
# use Binary Cross Entropy Loss
criterion = torch.nn.NLLLoss()

In [299]:
x_train_tensor = torch.from_numpy(X).float()
y_train_tensor = torch.from_numpy(Y).long()

In [322]:
EPOCHS = 100000

def train(model, optim, criterion, x, y, epochs=EPOCHS):
    for e in range(1, epochs + 1):
        optim.zero_grad()
        out = model(x)
        loss = criterion(out, y)
        loss.backward()
        optim.step()
        if e%1000==0:
            acc = torch.argmax(model.forward(x_train_tensor), axis=1)==y_train_tensor
            print(f"Loss at epoch {e}: {loss.data}, correct predicted: {np.sum(acc.numpy())}")
            
    return model

model = train(model, optim, criterion, x_train_tensor, y_train_tensor)

Loss at epoch 1000: 6.725808143615723, correct predicted: 1697
Loss at epoch 2000: 6.725768566131592, correct predicted: 1697
Loss at epoch 3000: 6.725728511810303, correct predicted: 1697
Loss at epoch 4000: 6.725689888000488, correct predicted: 1697
Loss at epoch 5000: 6.725651264190674, correct predicted: 1697
Loss at epoch 6000: 6.725613117218018, correct predicted: 1697
Loss at epoch 7000: 6.7255754470825195, correct predicted: 1697
Loss at epoch 8000: 6.725536823272705, correct predicted: 1697
Loss at epoch 9000: 6.725498676300049, correct predicted: 1697
Loss at epoch 10000: 6.725460529327393, correct predicted: 1697
Loss at epoch 11000: 6.725424289703369, correct predicted: 1697
Loss at epoch 12000: 6.725387096405029, correct predicted: 1697
Loss at epoch 13000: 6.7253499031066895, correct predicted: 1697
Loss at epoch 14000: 6.725313186645508, correct predicted: 1697
Loss at epoch 15000: 6.72527551651001, correct predicted: 1697
Loss at epoch 16000: 6.725240230560303, correct 

In [319]:
1675/x_train_tensor.shape[0]

0.4188547136784196

In [312]:
X_test2 = np.stack(X_test["transcription"].apply(word_vector))
Y_test2 = y_test.apply(f).to_numpy()

In [314]:
x_test_tensor = torch.from_numpy(X_test2).float()
y_test_tensor = torch.from_numpy(Y_test2).long()

In [323]:
acc = torch.argmax(model.forward(x_test_tensor), axis=1)==y_test_tensor
np.sum(acc.numpy())

243

In [316]:
x_test_tensor.shape

torch.Size([1000, 100])